In [1]:
import os
import sys
sys.path.append(os.chdir('../../'))

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import datetime

import matplotlib.pyplot as plt
import plotly.plotly as ply
import plotly.graph_objs as go
import src.utils.graph as graph
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

#from sklearn.multioutput import MultiOutputRegressor
from src.algorithms.multioutput import MultiOutputRegressor, RegressorChain

from sklearn.utils import shuffle
#from catboost import CatBoostRegressor
from src.algorithms.catboost_model import CatBoost

from sklearn.metrics import mean_absolute_error

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

import src.data as data
import src.utility as utility

In [3]:
#dataset_train = base_df.merge(data.base_dataset(), how='left')
X, Y = data.dataset('train', onehot=False, drop_index_columns=True)

caching merged dataset train


In [3]:
#path = os.path.join(f'resources/dataset/preprocessed/base_dataframe_train_inferred.csv.gz')
#dataset = pd.read_csv(path, parse_dates=True, index_col=0)
#X, Y = data.dataset_interpolated('train', onehot=False)

In [5]:
X.head(10)

,EMERGENCY_LANE,LANES,ROAD_TYPE,EVENT_DETAIL,EVENT_TYPE,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_SD_-4,SPEED_SD_-3,SPEED_SD_-2,SPEED_SD_-1,SPEED_MAX_-4,SPEED_MAX_-3,SPEED_MAX_-2,SPEED_MAX_-1,SPEED_MIN_-4,SPEED_MIN_-3,SPEED_MIN_-2,SPEED_MIN_-1,N_VEHICLES_-4,N_VEHICLES_-3,N_VEHICLES_-2,N_VEHICLES_-1,WEATHER_-4,WEATHER_-3,WEATHER_-2,WEATHER_-1,DISTANCE_-4,DISTANCE_-3,DISTANCE_-2,DISTANCE_-1,TEMPERATURE_-4,TEMPERATURE_-3,TEMPERATURE_-2,TEMPERATURE_-1,MIN_TEMPERATURE_-4,MIN_TEMPERATURE_-3,MIN_TEMPERATURE_-2,MIN_TEMPERATURE_-1,MAX_TEMPERATURE_-4,MAX_TEMPERATURE_-3,MAX_TEMPERATURE_-2,MAX_TEMPERATURE_-1,avg_speed_street,avg_speed_sd_street,avg_speed_min_street,avg_speed_max_street,avg_n_vehicles_street,avg_speed_sensor,avg_speed_sd_sensor,avg_speed_min_sensor,avg_speed_max_sensor,avg_n_vehicles_sensor,avg_speed_sensor_hour_y_0,avg_speed_sd_sensor_hour_y_0,avg_speed_min_sensor_hour_y_0,avg_speed_max_sensor_hour_y_0,avg_n_vehicles_sensor_hour_y_0,avg_speed_sensor_hour_y_1,avg_speed_sd_sensor_hour_y_1,avg_speed_min_sensor_hour_y_1,avg_speed_max_sensor_hour_y_1,avg_n_vehicles_sensor_hour_y_1,avg_speed_sensor_hour_y_2,avg_speed_sd_sensor_hour_y_2,avg_speed_min_sensor_hour_y_2,avg_speed_max_sensor_hour_y_2,avg_n_vehicles_sensor_hour_y_2,avg_speed_sensor_hour_y_3,avg_speed_sd_sensor_hour_y_3,avg_speed_min_sensor_hour_y_3,avg_speed_max_sensor_hour_y_3,avg_n_vehicles_sensor_hour_y_3,AVG_SPEED_EVENT,STD_SPEED_EVENT,SPEED_AVG
1554,1,3,1,25,Calamita_naturale,225.8283,228.5009,240.6095,222.6471,32.3762,28.2049,35.5813,27.4555,343.016,309.038,375.376,304.184,135.912,181.216,182.834,176.362,131.0,107.0,113.0,94.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,12.0,12.0,12.0,12.0,24.0,24.0,24.0,24.0,18.0,18.0,18.0,18.0,29.0,29.0,29.0,29.0,207.479240,26.208931,135.336730,288.928813,542.300278,207.477320,28.866409,129.954057,295.575019,559.798452,232.047949,33.551426,167.233783,330.206833,85.258333,230.225296,34.532635,160.856167,330.395600,85.716667,231.659763,34.256662,163.173261,333.212824,101.025210,229.926503,32.748503,161.301117,326.391050,113.508333,214.233770,29.078995,219.968881
1370,1,3,1,25,Calamita_naturale,232.1462,224.8530,237.0677,230.2363,33.9988,43.0886,26.9673,29.9258,310.656,315.510,339.780,347.870,74.428,84.136,177.980,177.980,88.0,66.0,79.0,64.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,12.0,12.0,12.0,12.0,24.0,24.0,24.0,24.0,18.0,18.0,18.0,18.0,29.0,29.0,29.0,29.0,207.479240,26.208931,135.336730,288.928813,542.300278,206.683304,26.106423,139.699447,291.509391,649.815637,225.280384,30.602174,160.884902,316.915803,79.803279,223.670739,31.033443,156.477983,319.013438,83.396694,225.784232,31.169377,158.365066,321.597393,97.295082,223.788538,30.082925,154.664885,320.642508,109.262295,214.233770,29.078995,219.968881
1745,1,3,1,25,Calamita_naturale,209.1308,208.2986,217.8098,210.9917,25.5567,27.5619,36.3047,33.1063,273.442,284.768,351.106,362.432,160.182,147.238,148.856,139.148,95.0,107.0,120.0,144.0,Temporale con Debole Pioggia,Temporale con Debole Pioggia,Nelle Vicinanze Tempesta di Polvere,Nelle Vicinanze Tempesta di Polvere,11.0,11.0,11.0,11.0,21.0,21.0,20.0,20.0,21.0,21.0,20.0,20.0,21.0,21.0,21.0,21.0,207.479240,26.208931,135.336730,288.928813,542.300278,202.721017,26.911213,133.360197,288.642584,758.930827,216.864556,34.251082,143.597500,319.029150,139.475000,218.316348,32.992870,146.145850,321.274125,142.662500,219.217052,34.042751,141.837925,321.415700,162.975000,218.636708,33.501840,140.300825,322.669650,219.000000,214.233770,29.078995,219.968881
1639,1,3,1,25,Calamita_naturale,189.9103,213.9138,225.2186,218.1234,46.2343,21.7366,27.6684,24.5135,257.262,278.296,309.038,292.858,69.574,160.182,129.440,155.328,83.0,91.0,92.0,95.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,13.0,13.0,13.0,13.0,24.0,24.0,24.0,24.0,18.0,18.0,18.0,18.0,29.0,29.0,29.0,29.0,207.479240,26.208931,135.336730,288.928813,542.300278,200.865451,25.770041,121.826117,282.482585,713.954422,222.933027,31.943704,134.174639,315.

In [4]:
print(X.shape)
print(Y.shape)
print()
print(Y.dtypes)

(33202, 77)
(33202, 4)

SPEED_AVG_Y_0    float64
SPEED_AVG_Y_1    float64
SPEED_AVG_Y_2    float64
SPEED_AVG_Y_3    float64
dtype: object


## Preprocess data

In [4]:
X.head(10)

,EMERGENCY_LANE,LANES,ROAD_TYPE,EVENT_DETAIL,EVENT_TYPE,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_SD_-4,SPEED_SD_-3,SPEED_SD_-2,SPEED_SD_-1,SPEED_MAX_-4,SPEED_MAX_-3,SPEED_MAX_-2,SPEED_MAX_-1,SPEED_MIN_-4,SPEED_MIN_-3,SPEED_MIN_-2,SPEED_MIN_-1,N_VEHICLES_-4,N_VEHICLES_-3,N_VEHICLES_-2,N_VEHICLES_-1,WEATHER_-4,WEATHER_-3,WEATHER_-2,WEATHER_-1,DISTANCE_-4,DISTANCE_-3,DISTANCE_-2,DISTANCE_-1,TEMPERATURE_-4,TEMPERATURE_-3,TEMPERATURE_-2,TEMPERATURE_-1,MIN_TEMPERATURE_-4,MIN_TEMPERATURE_-3,MIN_TEMPERATURE_-2,MIN_TEMPERATURE_-1,MAX_TEMPERATURE_-4,MAX_TEMPERATURE_-3,MAX_TEMPERATURE_-2,MAX_TEMPERATURE_-1
0,1,2,1,11,Veicolo_in_avaria,NaN,199.6642,199.3150,204.7759,NaN,41.4457,52.0643,43.0659,NaN,263.734,292.858,294.476,NaN,59.866,59.866,59.866,NaN,107.0,129.0,139.0,NaN,Quasi Sereno,Quasi Sereno,Quasi Sereno,NaN,38.0,38.0,38.0,NaN,28.0,28.0,28.0,NaN,21.0,21.0,21.0,NaN,29.0,29.0,29.0
1,1,2,1,6,Gestione_viabilita,NaN,NaN,NaN,208.9124,NaN,NaN,NaN,38.8807,NaN,NaN,NaN,352.724,NaN,NaN,NaN,59.866,NaN,NaN,NaN,238.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2,1,6,Gestione_viabilita,NaN,NaN,208.9124,201.9654,NaN,NaN,38.8807,39.7029,NaN,NaN,352.724,286.386,NaN,NaN,59.866,59.866,NaN,NaN,238.0,216.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2,1,6,Gestione_viabilita,NaN,NaN,NaN,213.5957,NaN,NaN,NaN,35.7280,NaN,NaN,NaN,334.926,NaN,NaN,NaN,59.866,NaN,NaN,NaN,246.0,NaN,NaN,NaN,Rovescio Nelle Vicinanze con Pioggia,NaN,NaN,NaN,38.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,19.0,NaN,NaN,NaN,21.0
4,1,2,1,2,Manutenzione_opere_in_verde,210.7883,213.6824,223.9108,223.1689,34.5211,33.1529,32.9331,30.7040,281.532,343.016,389.938,341.398,59.866,61.484,64.720,59.866,231.0,228.0,222.0,239.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,226.0,226.0,38.0,38.0,21.0,21.0,19.0,19.0,10.0,10.0,12.0,12.0,21.0,21.0,19.0,19.0
5,1,2,1,2,Manutenzione_opere_in_verde,223.1689,218.5277,219.2802,214.6672,30.7040,29.5927,28.9895,32.1558,341.398,328.454,300.948,286.386,59.866,59.866,61.484,59.866,239.0,265.0,255.0,258.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,38.0,38.0,38.0,38.0,19.0,19.0,19.0,19.0,12.0,12.0,12.0,12.0,19.0,19.0,19.0,19.0
6,1,2,1,2,Manutenzione_opere_in_verde,218.5277,219.2802,214.6672,216.8678,29.5927,28.9895,32.1558,29.6041,328.454,300.948,286.386,310.656,59.866,61.484,59.866,59.866,265.0,255.0,258.0,232.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,38.0,38.0,38.0,38.0,19.0,19.0,19.0,19.0,12.0,12.0,12.0,12.0,19.0,19.0,19.0,19.0
7,1,2,1,11,Veicolo_in_avaria,NaN,NaN,199.6642,199.3150,NaN,NaN,41.4457,52.0643,NaN,NaN,263.734,292.858,NaN,NaN,59.866,59.866,NaN,NaN,107.0,129.0,NaN,NaN,Quasi Sereno,Quasi Sereno,NaN,NaN,38.0,38.0,NaN,NaN,28.0,28.0,NaN,NaN,21.0,21.0,NaN,NaN,29.0,29.0
8,1,2,1,2,Manutenzione_opere_in_verde,NaN,216.4407,208.1131,217.6035,NaN,44.8391,42.4317,37.3565,NaN,368.904,281.532,349.488,NaN,59.866,59.866,59.866,NaN,183.0,186.0,231.0,NaN,Quasi Sereno,Quasi Sereno,Quasi Sereno,NaN,38.0,38.0,38.0,NaN,12.0,12.0,14.0,NaN,13.0,13.0,12.0,NaN,14.0,14.0,14.0
9,1,2,1,2,Manutenzione_opere_in_verde,211.3166,212.3555,207.6177,213.4945,27.7924,37.2444,34.9689,33.5346,278.296,307.420,273.442,296.094,61.484,59.866,59.866,59.866,280.0,232.0,274.0,258.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,38.0,38.0,226.0,226.0,15.0,15.0,17.0,17.0,12.0,12.0,10.0,10.0,14.0,14.0,15.0,15.0


In [7]:
# add features
import src.preprocessing.other_features as feat
avg_speed_road_event = feat.avg_speed_for_roadtype_event()
X = X.merge(avg_speed_road_event, how='left', on=['EVENT_TYPE','ROAD_TYPE'])

avg_speed_road_event.head()

In [4]:
X.fillna(0, inplace=True)
#Y.fillna(0, inplace=True)

weather_cols = [col for col in X.columns if col.startswith('WEATHER_')]
categorical_cols = ['EMERGENCY_LANE', 'ROAD_TYPE', 'EVENT_DETAIL','EVENT_TYPE',] + weather_cols

## Train/Validation split

In [12]:
#X, Y = shuffle(X, Y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

In [6]:
def evaluate(model, X_test, y_test):
    mask_test = np.all(y_test.notnull(), axis=1)
    print('number of valid samples:', (mask_test*1).sum())

    y_pred = model.predict(X_test[mask_test])
    return mean_absolute_error(y_test[mask_test], y_pred)

## Fit

In [17]:
from catboost import CatBoostRegressor

mae = []
for colY in Y.columns:
    mask = Y[colY].notnull()
    x = X_train[mask]
    y = y_train[colY][mask]
    
    mask_val = y_test[colY].notnull()
    x_val = X_test[mask]
    y_val = y_test[colY][mask_val]
    
    catboost = CatBoostRegressor(**{
        'loss_function': 'MAE',
        'eval_metric': 'MAE',
        'iterations':2000,
        'depth':6,
        'learning_rate':1,
        'early_stopping_rounds': 80,
        'cat_features': categorical_cols,
        'verbose': True
    })
    
    catboost.fit(x, y, eval_set=[(x_val, y_val)])
    
    # evaluate
    y_pred = catboost.predict(x_val)
    
    mae.append( mean_absolute_error(y_val, y_pred) )

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0:	learn: 177.7051518	test: 183.3838661	best: 183.3838661 (0)	total: 64.2ms	remaining: 2m 8s
1:	learn: 177.2066703	test: 182.8855700	best: 182.8855700 (1)	total: 134ms	remaining: 2m 13s
2:	learn: 176.7080109	test: 182.3863849	best: 182.3863849 (2)	total: 209ms	remaining: 2m 19s
3:	learn: 176.2095406	test: 181.8884537	best: 181.8884537 (3)	total: 287ms	remaining: 2m 23s
4:	learn: 175.7110258	test: 181.3898264	best: 181.3898264 (4)	total: 360ms	remaining: 2m 23s
5:	learn: 175.2125470	test: 180.8924187	best: 180.8924187 (5)	total: 430ms	remaining: 2m 22s
6:	learn: 174.7142695	test: 180.3983990	best: 180.3983990 (6)	total: 511ms	remaining: 2m 25s
7:	learn: 174.2153597	test: 179.8998603	best: 179.8998603 (7)	total: 576ms	remaining: 2m 23s
8:	learn: 173.7174475	test: 179.4053919	best: 179.4053919 (8)	total: 651ms	remaining: 2m 23s
9:	learn: 173.2189454	test: 178.9068115	best: 178.9068115 (9)	total: 718ms	remaining: 2m 22s
10:	learn: 172.7208487	test: 178.4085969	best: 178.4085969 (10)	total:

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0:	learn: 178.1178345	test: 183.6057419	best: 183.6057419 (0)	total: 57.3ms	remaining: 1m 54s
1:	learn: 177.6200324	test: 183.1065959	best: 183.1065959 (1)	total: 117ms	remaining: 1m 56s
2:	learn: 177.1205405	test: 182.6070087	best: 182.6070087 (2)	total: 155ms	remaining: 1m 43s
3:	learn: 176.6215992	test: 182.1077964	best: 182.1077964 (3)	total: 212ms	remaining: 1m 45s
4:	learn: 176.1230408	test: 181.6090495	best: 181.6090495 (4)	total: 277ms	remaining: 1m 50s
5:	learn: 175.6239486	test: 181.1108092	best: 181.1108092 (5)	total: 330ms	remaining: 1m 49s
6:	learn: 175.1243486	test: 180.6119179	best: 180.6119179 (6)	total: 365ms	remaining: 1m 44s
7:	learn: 174.6245850	test: 180.1121434	best: 180.1121434 (7)	total: 387ms	remaining: 1m 36s
8:	learn: 174.1263118	test: 179.6192451	best: 179.6192451 (8)	total: 438ms	remaining: 1m 36s
9:	learn: 173.6264302	test: 179.1193639	best: 179.1193639 (9)	total: 454ms	remaining: 1m 30s
10:	learn: 173.1274715	test: 178.6201117	best: 178.6201117 (10)	total

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0:	learn: 178.2720632	test: 183.6882458	best: 183.6882458 (0)	total: 57.5ms	remaining: 1m 54s
1:	learn: 177.7745480	test: 183.1911266	best: 183.1911266 (1)	total: 116ms	remaining: 1m 55s
2:	learn: 177.2775558	test: 182.6960076	best: 182.6960076 (2)	total: 170ms	remaining: 1m 53s
3:	learn: 176.7798193	test: 182.2001034	best: 182.2001034 (3)	total: 223ms	remaining: 1m 51s
4:	learn: 176.2814143	test: 181.7015984	best: 181.7015984 (4)	total: 287ms	remaining: 1m 54s
5:	learn: 175.7825381	test: 181.2045834	best: 181.2045834 (5)	total: 341ms	remaining: 1m 53s
6:	learn: 175.2840116	test: 180.7057302	best: 180.7057302 (6)	total: 394ms	remaining: 1m 52s
7:	learn: 174.7852006	test: 180.2066894	best: 180.2066894 (7)	total: 446ms	remaining: 1m 51s
8:	learn: 174.2872730	test: 179.7097339	best: 179.7097339 (8)	total: 506ms	remaining: 1m 51s
9:	learn: 173.7882992	test: 179.2122012	best: 179.2122012 (9)	total: 564ms	remaining: 1m 52s
10:	learn: 173.2887831	test: 178.7132252	best: 178.7132252 (10)	total

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0:	learn: 178.3600737	test: 183.8659000	best: 183.8659000 (0)	total: 54.2ms	remaining: 1m 48s
1:	learn: 177.8614294	test: 183.3669732	best: 183.3669732 (1)	total: 113ms	remaining: 1m 52s
2:	learn: 177.3632222	test: 182.8696069	best: 182.8696069 (2)	total: 167ms	remaining: 1m 51s
3:	learn: 176.8644544	test: 182.3708401	best: 182.3708401 (3)	total: 222ms	remaining: 1m 50s
4:	learn: 176.3670827	test: 181.8730577	best: 181.8730577 (4)	total: 279ms	remaining: 1m 51s
5:	learn: 175.8691310	test: 181.3810003	best: 181.3810003 (5)	total: 336ms	remaining: 1m 51s
6:	learn: 175.3704847	test: 180.8832939	best: 180.8832939 (6)	total: 389ms	remaining: 1m 50s
7:	learn: 174.8722230	test: 180.3840807	best: 180.3840807 (7)	total: 444ms	remaining: 1m 50s
8:	learn: 174.3749322	test: 179.8856283	best: 179.8856283 (8)	total: 504ms	remaining: 1m 51s
9:	learn: 173.8761420	test: 179.3869698	best: 179.3869698 (9)	total: 559ms	remaining: 1m 51s
10:	learn: 173.3777698	test: 178.8943586	best: 178.8943586 (10)	total

In [18]:
mae

[6.6273399128792825, 7.877166646134302, 9.378274779374266, 10.505643840577255]

In [22]:
sum(mae)/4

8.597106294741277

In [6]:
catboost = CatBoost(params_dict={
    'X': X,
    'loss_function': 'MAE',
    'eval_metric': 'MAE',
    'iterations':2,
    'depth':6,
    'learning_rate':1,
    'cat_features': categorical_cols,
    'verbose': False
})

In [8]:
model = RegressorChain(catboost)
model.fit(X, Y)

catboost: fitting...
catboost: fitting...
catboost: fitting...
catboost: fitting...


RegressorChain(base_estimator=<src.algorithms.catboost_model.CatBoost object at 0x1a20daf3c8>,
        cv=None, order=None, random_state=None)

In [11]:
evaluate(X_test, y_test)

7.8075137571574285

## Baseline

Catboost chain (no features + weather): 7.076924911258332 - TEST: 7.9718490219958404

Catboost chain: 7.8075137571574285

Catboost chain (avg speed x road type): 7.096708023412791

Catboost chain (avg speed x road type + avg speed sensor + avg_speed_road): 6.79054199721705

## Test model

In [3]:
import src.algorithms.inout as inout

In [4]:
X, Y = data.dataset('test', onehot=False, drop_index_columns=True)
X.fillna(0, inplace=True)
X.shape

caching merged dataset test


(2981, 78)

In [5]:
X.head()

,EMERGENCY_LANE,LANES,ROAD_TYPE,EVENT_DETAIL,EVENT_TYPE,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_SD_-4,SPEED_SD_-3,SPEED_SD_-2,SPEED_SD_-1,SPEED_MAX_-4,SPEED_MAX_-3,SPEED_MAX_-2,SPEED_MAX_-1,SPEED_MIN_-4,SPEED_MIN_-3,SPEED_MIN_-2,SPEED_MIN_-1,N_VEHICLES_-4,N_VEHICLES_-3,N_VEHICLES_-2,N_VEHICLES_-1,WEATHER_-4,WEATHER_-3,WEATHER_-2,WEATHER_-1,DISTANCE_-4,DISTANCE_-3,DISTANCE_-2,DISTANCE_-1,TEMPERATURE_-4,TEMPERATURE_-3,TEMPERATURE_-2,TEMPERATURE_-1,MIN_TEMPERATURE_-4,MIN_TEMPERATURE_-3,MIN_TEMPERATURE_-2,MIN_TEMPERATURE_-1,MAX_TEMPERATURE_-4,MAX_TEMPERATURE_-3,MAX_TEMPERATURE_-2,MAX_TEMPERATURE_-1,avg_speed_street,avg_speed_sd_street,avg_speed_min_street,avg_speed_max_street,avg_n_vehicles_street,avg_speed_sensor,avg_speed_sd_sensor,avg_speed_min_sensor,avg_speed_max_sensor,avg_n_vehicles_sensor,avg_speed_sensor_hour_y_0,avg_speed_sd_sensor_hour_y_0,avg_speed_min_sensor_hour_y_0,avg_speed_max_sensor_hour_y_0,avg_n_vehicles_sensor_hour_y_0,avg_speed_sensor_hour_y_1,avg_speed_sd_sensor_hour_y_1,avg_speed_min_sensor_hour_y_1,avg_speed_max_sensor_hour_y_1,avg_n_vehicles_sensor_hour_y_1,avg_speed_sensor_hour_y_2,avg_speed_sd_sensor_hour_y_2,avg_speed_min_sensor_hour_y_2,avg_speed_max_sensor_hour_y_2,avg_n_vehicles_sensor_hour_y_2,avg_speed_sensor_hour_y_3,avg_speed_sd_sensor_hour_y_3,avg_speed_min_sensor_hour_y_3,avg_speed_max_sensor_hour_y_3,avg_n_vehicles_sensor_hour_y_3,AVG_SPEED_EVENT,STD_SPEED_EVENT,SPEED_AVG
2749,0,1,0,8,Ostacolo_in_carreggiata,164.9204,162.1236,166.6540,157.5683,11.9988,13.2142,13.1212,17.4809,187.688,179.598,187.688,195.778,152.092,152.092,144.002,126.204,14.0,5.0,18.0,26.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,14.0,14.0,14.0,14.0,4.0,4.0,4.0,4.0,1.0,1.0,1.0,1.0,6.0,6.0,6.0,6.0,175.772919,25.274510,131.042390,221.595329,26.335432,153.856952,17.761712,118.600734,190.311964,44.811875,161.851465,16.676040,139.032429,187.052357,10.482143,161.120902,17.341626,137.573730,187.265279,9.504505,161.332878,16.075892,140.450991,183.864938,7.938053,161.995596,16.063774,142.009463,183.538130,7.129630,169.289942,27.424423,180.747671
1335,0,2,2,56,Meteo,196.1016,226.2889,192.1375,194.6223,20.1049,29.7050,28.4664,24.3725,218.430,273.442,228.138,232.992,165.036,190.924,155.328,161.800,10.0,14.0,16.0,7.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,55.0,55.0,55.0,55.0,4.0,4.0,4.0,4.0,1.0,1.0,1.0,1.0,6.0,6.0,6.0,6.0,212.345308,29.881775,137.218459,277.373949,40.919547,196.513177,26.592639,130.302709,246.671964,37.404051,192.365827,28.639844,161.217520,228.105640,5.410000,196.218940,28.199220,164.653247,226.572194,4.903226,196.751107,24.837732,169.940563,221.480604,3.750000,195.676687,23.776495,169.389588,220.798619,4.237113,0.000000,0.000000,208.662555
1313,0,2,2,56,Meteo,240.3352,225.6634,218.0255,284.7680,33.1693,35.2651,38.4380,7.0527,299.330,288.004,307.420,292.858,177.980,160.182,173.126,279.914,26.0,34.0,24.0,3.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,48.0,48.0,48.0,48.0,4.0,4.0,4.0,4.0,1.0,1.0,1.0,1.0,6.0,6.0,6.0,6.0,212.345308,29.881775,137.218459,277.373949,40.919547,227.925257,33.118520,144.024051,307.586899,44.379046,229.581576,32.330252,190.108085,277.604547,7.102564,231.855774,33.744251,187.602842,278.523088,6.675439,229.903964,31.451609,190.203378,270.967412,6.168067,227.585155,31.654241,185.465009,271.936557,7.330435,0.000000,0.000000,208.662555
1605,0,2,0,56,Meteo,270.4371,201.6028,0.0000,0.0000,37.0641,12.6070,0.0000,0.0000,330.072,220.048,0.000,0.000,210.340,186.070,0.000,0.000,14.0,10.0,0.0,0.0,Quasi Sereno,Quasi Sereno,0,0,80.0,80.0,0.0,0.0,6.0,6.0,0.0,0.0,5.0,5.0,0.0,0.0,13.0,13.0,0.0,0.0,214.648044,32.363169,154.367700,295.841913,42.744288,216.955383,27.561166,164.702300,284.218959,42.984394,218.540623,26.648928,190.568396,251.999055,4.725275,209.986658,27.746478,182.154840,244.877309,4.790123,211.680883,24.225600,186.417957,240.194710,4.193548,212.960979,24.128422,185.897204,244.663592,5.087379,144.279724,19.480299,180.747671
1608,0,2,0,56,Meteo,212.5648,0

In [6]:
#model = inout.load('saved_models/catboost_multiout_5.98269_baseline_5ft.jl')
model = inout.load('saved_models/catboost_multiout_8.45851_baseline.jl')

In [7]:
inout.evaluate(model, X_test=X, y_test=Y)

This evaluation will be done for samples having ALL y not null!


(8.352709228393282,
 [7.060148226263454, 8.086440020790906, 8.995492885807689, 9.268755780711079])